<a href="https://colab.research.google.com/github/whyu38/analisis-sentimen-project/blob/main/Pra_Analisis_Sentimen_Wahyu_Sukma_J.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Library

In [ ]:
!pip install google-play-scraper
!pip install Sastrawi # Opsional untuk stemming bahasa Indonesia

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 12.5 MB/s eta 0:00:00


# Import Library

In [ ]:
from google_play_scraper import Sort, reviews
import pandas as pd
import numpy as np
import re
import string
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

# Data Preparation

In [ ]:
APP_ID = 'com.gojek.app'
LANG = 'id'
COUNTRY = 'id'
COUNT = 12000

# Proses Scraping
result, continuation_token = reviews(
    APP_ID,
    lang=LANG,
    country=COUNTRY,
    sort=Sort.NEWEST,
    count=COUNT,
    filter_score_with=None
)

# Konversi ke DataFrame
df = pd.DataFrame(result)
df = df[['content', 'score']]

# Simpan hasil scraping (PENTING untuk dikumpulin)
df.to_csv('hasil_scraping.csv', index=False)
print(f"Berhasil mendapatkan {len(df)} data.")
df.head()

Berhasil mendapatkan 12000 data.


,content,score
0,"ini kalo di gojek notip nya suka telat, tolong...",3
1,"pesan gofood, bahan baku resto ga ada. resto d...",1
2,saat ini saya mendapatkan pelayanan yang sanga...,1
3,aplikasi ini sangat berguna,5
4,agak sedeng app gojek. gw pk app ni dah dr lam...,1


# Data Labeling

In [ ]:
def pelabelan(score):
  if score < 3:
    return 'Negatif'
  elif score == 3:
    return 'Netral'
  else:
    return 'Positif'

df['sentiment'] = df['score'].apply(pelabelan)

# Cek sebaran data
print(df['sentiment'].value_counts())

sentiment
Positif    7625
Negatif    3922
Netral      453
Name: count, dtype: int64


# Data Preprocessing

In [ ]:
def cleaning_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = text.strip()
    return text

df['clean_text'] = df['content'].apply(cleaning_text)

# Data Splitting & Feature Extraction

In [ ]:
# 1. Encoding Label (Negatif, Netral, Positif -> 0, 1, 2)
le = LabelEncoder()
y = le.fit_transform(df['sentiment'])
y = to_categorical(y)

# 2. Split Data (Wajib ada Training dan Testing)
X_train, X_test, y_train, y_test = train_test_split(df['clean_text'], y, test_size=0.2, random_state=42)

# 3. Tokenisasi
tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

# Konversi teks ke sequence angka
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Padding (agar panjang input sama)
X_train_pad = pad_sequences(X_train_seq, maxlen=100, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=100, padding='post', truncating='post')

# Model Training

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=128, input_length=100),
    tf.keras.layers.LSTM(64, dropout=0.2, recurrent_dropout=0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax') # 3 neuron karena 3 kelas
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Callbacks untuk menghentikan training jika akurasi sudah bagus (opsional)
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.92 and logs.get('val_accuracy')>0.92):
      print("\nAkurasi telah mencapai >92%!")
      self.model.stop_training = True

callbacks = myCallback()

# Latih Model
history = model.fit(
    X_train_pad, y_train,
    epochs=20,
    validation_data=(X_test_pad, y_test),
    batch_size=128,
    callbacks=[callbacks],
    verbose=2
)

Epoch 1/20


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


75/75 - 29s - 386ms/step - accuracy: 0.6290 - loss: 0.8053 - val_accuracy: 0.6342 - val_loss: 0.7789
Epoch 2/20
75/75 - 16s - 216ms/step - accuracy: 0.6357 - loss: 0.7790 - val_accuracy: 0.6342 - val_loss: 0.7789
Epoch 3/20
75/75 - 16s - 212ms/step - accuracy: 0.6357 - loss: 0.7793 - val_accuracy: 0.6342 - val_loss: 0.7795
Epoch 4/20
75/75 - 16s - 208ms/step - accuracy: 0.6357 - loss: 0.7785 - val_accuracy: 0.6342 - val_loss: 0.7790
Epoch 5/20
75/75 - 15s - 206ms/step - accuracy: 0.6357 - loss: 0.7795 - val_accuracy: 0.6342 - val_loss: 0.7793
Epoch 6/20
75/75 - 21s - 276ms/step - accuracy: 0.6357 - loss: 0.7795 - val_accuracy: 0.6342 - val_loss: 0.7795
Epoch 7/20
75/75 - 15s - 205ms/step - accuracy: 0.6357 - loss: 0.7783 - val_accuracy: 0.6342 - val_loss: 0.7795
Epoch 8/20
75/75 - 21s - 273ms/step - accuracy: 0.6357 - loss: 0.7785 - val_accuracy: 0.6342 - val_loss: 0.7802
Epoch 9/20
75/75 - 20s - 272ms/step - accuracy: 0.6358 - loss: 0.7788 - val_accuracy: 0.6342 - val_loss: 0.7787
Epo

# Inference

In [ ]:
input_text = ["Aplikasi ini sangat bagus, membantu banget!", "Drivernya ramah"]
seq = tokenizer.texts_to_sequences(input_text)
padded = pad_sequences(seq, maxlen=100, padding='post', truncating='post')

prediction = model.predict(padded)
import numpy as np

# Translate prediksi kembali ke label
labels = le.classes_ # ['Negatif', 'Netral', 'Positif']
for i, text in enumerate(input_text):
    print(f"Teks: {text}\nPrediksi: {labels[np.argmax(prediction[i])]}\n")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
Teks: Aplikasi ini sangat bagus, membantu banget!
Prediksi: Positif

Teks: Drivernya ramah
Prediksi: Positif

